In [1]:
import pathlib
import json
from collections import Counter
from IPython.display import clear_output

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
js_files = list(pathlib.Path(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\official').iterdir())
for i,js in enumerate(js_files):
    print(i)
    official_label = json.loads(js.read_text(encoding='utf8'))
    H = int(official_label['imageHeight'])
    W = int(official_label['imageWidth'])
    
    bboxes = []
    for shape in official_label['shapes']:
        points = np.array(shape['points'])
        points[:,0] = np.clip(points[:,0],0,W)
        points[:,1] = np.clip(points[:,1],0,H)        
        x,y,w,h = cv2.boundingRect(points)
        
        x,w = x/W, w/W
        y,h = y/H, h/H
        x = x+w/2
        y = y+h/2
        points = points/np.array([W,H])
        
        bbox = {}
        bbox['text'] = shape['label']
        bbox['group'] = shape['group_id']
        bbox['points'] = points.tolist()
        bbox['center'] = points.mean(axis=0).tolist()
        bbox['xywh'] = [x,y,w,h]
        bboxes.append(bbox)
        
    bboxes = sorted(bboxes,key = lambda x: x['center'])
    
    my_label = {}
    my_label['ImageName'] = official_label['imagePath']
    my_label['ImageHeight'] = H
    my_label['ImageWidth'] = W
    my_label['bboxes'] = bboxes
    my_label['bboxes_count'] = len(bboxes)
    
    s = json.dumps(my_label)
    pathlib.Path(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\custom').joinpath(js.name).write_text(s)
    clear_output(wait=True)
print('Finish')

Finish


In [ ]:
irr = []
table = {}
idx = 0
for i,js in enumerate(js_files):
    print(i)
    label = json.loads(js.read_text(encoding='utf8'))
    H = int(label['imageHeight'])
    W = int(label['imageWidth'])
    for bbox in label['shapes']:
        (x0,y0),(x1,y1),(x2,y2),(x3,y3) = bbox['points']
        
        try:
            assert 0<=x0<=W
            assert 0<=x1<=W
            assert 0<=x2<=W
            assert 0<=x3<=W

            assert 0<=y0<=H
            assert 0<=y1<=H
            assert 0<=y2<=H
            assert 0<=y3<=H
        except Exception:
            irr.append(label)

        x_lu,y_lu,w,h = cv2.boundingRect(np.array(bbox['points']))
        determine = max(w,h)
        scaled_determine = max(w*1024/W,h*1024/H)
        
        table[idx] = {
            'image':label['imagePath'],
            'H':H,
            'W':W,
            'group':bbox['group_id'],
            'text':bbox['label'],
            'x0':x0,'y0':y0,'x1':x1,'y1':y1,'x2':x2,'y2':y2,'x3':x3,'y3':y3,
            'x_lu':x_lu,'y_lu':y_lu,'h':h,'w':w,
            'determine':determine,'scaled_determine':scaled_determine
        }
        idx += 1
    clear_output(wait=True)
table = pd.DataFrame.from_dict(table).T
print('Finish')

In [ ]:
len(irr)

In [ ]:
table.loc[table['group']!=1].groupby('image').count()['group'].sort_values(ascending=False)

In [ ]:
table['scaled_determine'].min()

In [ ]:
plt.figure(figsize=(20,10))
table['scaled_determine'].hist(bins=1000)

In [ ]:
table.loc[table['scaled_determine'] == 1025]

In [ ]:
table['scaled_determine'].max()

In [ ]:
img = plt.imread(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\official\img_2908.jpg')

In [ ]:
label = json.loads(pathlib.Path(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\img_2908.json').read_text(encoding='utf8'))

In [ ]:
img = cv2.polylines(img,np.array([x['points'] for x in label['shapes']]),True,(0,255,255),1)

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(img)

In [ ]:
#the image with most bboxes
img = plt.imread(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\official\img_7626.jpg')

In [ ]:
label = json.loads(pathlib.Path(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\img_7626.json').read_text(encoding='utf8'))

In [ ]:
img = cv2.polylines(img,np.array([x['points'] for x in label['shapes'] if ('' == x['label'])]),True,(255,0,255),1)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)

In [ ]:
max(bboxes.keys())

In [ ]:
len(corpus)

In [ ]:
len(image_shape)

In [ ]:
pd.options.display.max_rows=3700

In [ ]:
corpus_series = pd.Series(corpus).sort_values(ascending=False)

In [ ]:
import re

In [ ]:
symbol_index = corpus_series.index.map(lambda x:bool(re.match('[^\u4e00-\u9fa5a-zA-Z0-9]',x)))

In [ ]:
corpus_series.loc[symbol_index]